In [7]:
import pickle
import sys

if '..' not in sys.path:
    sys.path.append('..')
    
from main_fit_vectorizer import TFIDF_Vectorizer

In [8]:
with open('../Resource/vectorizer.pickle', 'rb') as pf:
    vectorizer = pickle.load(pf)

In [9]:
vectorizer_desc = vectorizer.vectorize_desc

In [10]:
print(vectorizer_desc.vocabulary_)

{'': 0, 'ปฏิบัติงานใน': 1796, 'law': 579, 'ส่งเสริมการ': 2297, 'ตาราง': 1575, 'ขั้นตอนการ': 1255, 'should': 927, 'microsoft': 644, 'ที่ต้องการ': 1691, 'ลา': 2103, 'hire': 490, 'รถยนต์ได้': 2018, 'admin': 59, 'สาขาช่าง': 2222, 'monday': 654, 'เบื้องต้นได้': 2505, 'connect': 245, 'งานด้าน': 1391, 'ความสะดวก': 1336, 'ผู้อื่น': 1888, 'เอกสาร': 2584, 'met': 641, 'ส': 2165, 'your': 1107, 'เวอร์': 2568, 'การบันทึก': 1180, 'estim': 375, 'ภายใต้สภาวะ': 1942, 'interest': 540, 'method': 642, 'ความคิดสร้างสรรค์': 1305, 'งานได้': 1406, 'compon': 233, 'establish': 373, 'ของสินค้า': 1245, 'java': 560, 'consolid': 249, 'suppli': 980, 'ประชุม': 1816, 'เคย': 2444, 'translat': 1037, 'moo': 657, 'import': 512, 'budget': 165, 'ขับขี่': 1251, 'who': 1091, 'สัมพันธ์': 2212, 'รับสมัคร': 2062, 'ขายเครื่องใช้ไฟฟ้า': 1263, 'sens': 912, 'outstand': 716, 'asset': 115, 'with': 1096, 'find': 414, 'creat': 272, 'most': 659, 'สื่อสารได้': 2283, 'เวลาการ': 2564, 'ว': 2120, 'โครงสร้าง': 2680, 'พบลูกค้า': 1906, 'ยื่น': 2

In [11]:
print(len(vectorizer_desc.vocabulary_))

2815


In [13]:
vectorizer_title = vectorizer.vectorize_title
print(len(vectorizer_title.vocabulary_))

373
